In [18]:
using Distributions
using ReactiveMP
using Rx

import Base: show

In [26]:
mutable struct InferenceActor <: Actor{AbstractMessage}
    index :: Int
    size  :: Int
    data  :: Vector{Float64}
    means :: Vector{Float64}
    
    y          :: ObservedVariable
    e_mean     :: EstimatedVariable
    e_variance :: EstimatedVariable
    
    InferenceActor(data::Vector{Float64}, y::ObservedVariable, e_mean::EstimatedVariable, e_variance::EstimatedVariable) = begin
        size  = length(data)
        means = Vector{Float64}(undef, size)
        return new(1, size, data, means, y, e_mean, e_variance)
    end
end

function update!(actor::InferenceActor, mean::Float64, variance::Float64)
    next!(actor.y.values, actor.data[actor.index])
    next!(actor.e_mean.values, mean)
    next!(actor.e_variance.values, variance)
end

function stop!(actor::InferenceActor)
    complete!(actor.y.values)
    complete!(actor.e_mean.values)
    complete!(actor.e_variance.values)
end

function Rx.on_next!(actor::InferenceActor, data::AbstractMessage)
    m = mean(data.distribution)
    v = var(data.distribution)
    
    actor.means[actor.index] = m
    
    actor.index += 1
    
    @async begin
        if actor.index < actor.size 
            update!(actor, m, v)
        else
            stop!(actor)
        end
    end
end

Rx.on_error!(actor::InferenceActor, err) = error(err)
Rx.on_complete!(actor::InferenceActor) = println("Completed: $(time())")

Base.show(io::IO, actor::InferenceActor) = print(io, "InferenceActor")

In [34]:
xPrevAddConstantNode = AdditionNode("xPrevAddConstantNode");
addConstant = ConstantVariable("addConstant", 1.0, xPrevAddConstantNode.in2);

estimatingXNode    = GaussianMeanVarianceFactorNode("estimatingXWithGaussianNode");
estimatedXMean     = EstimatedVariable("estimatedXMean", estimatingXNode.mean);
estimatedXVariance = EstimatedVariable("estimatedXVariance", estimatingXNode.variance);

xPrev = RandomVariable("xPrev", estimatingXNode.value, xPrevAddConstantNode.in1);

noiseNode     = GaussianMeanVarianceFactorNode("noiseNode");
noiseMean     = ConstantVariable("noiseMean", 0.0, noiseNode.mean);
noiseVariance = ConstantVariable("noiseVariance", 1.0, noiseNode.variance);

addXAndNoiseNode = AdditionNode("addXAndNoiseNode");

x = RandomVariable("x", xPrevAddConstantNode.out, addXAndNoiseNode.in1);
n = RandomVariable("n", noiseNode.value, addXAndNoiseNode.in2);
y = ObservedVariable("y", addXAndNoiseNode.out);

In [35]:
n = 10000
data = [ i for i::Float64 in 1:n ] + rand(Normal(0.0, 1.0), n);

In [36]:
actor = InferenceActor(data, y, estimatedXMean, estimatedXVariance);

In [37]:
function try_reactive(actor::InferenceActor, x)
    println(time())
    subscribe!(inference(x), actor)
    update!(actor, 0.0, 1000.0)
    return actor
end

try_reactive (generic function with 1 method)

In [38]:
actor = try_reactive(actor, x);

1.577127052835304e9
Completed: 1.577127054135495e9
